In [1]:
# Load needed packages
library(caTools)
library(ROCR)
library(Hmisc)
library(neuralnet)

Loading required package: gplots

Attaching package: ‘gplots’

The following object is masked from ‘package:stats’:

    lowess

Loading required package: lattice
Loading required package: survival
Loading required package: Formula
Loading required package: ggplot2

Attaching package: ‘Hmisc’

The following objects are masked from ‘package:base’:

    format.pval, units


Attaching package: ‘neuralnet’

The following object is masked from ‘package:ROCR’:

    prediction



In [2]:
df <- read.csv("Datasets/HEMQ.csv",na.strings=c("NA",""))

In [3]:
# Impute missing values
df$VALUE <- with(df,impute(VALUE,mean))
df$REASON <- with(df,impute(REASON,max))
df$JOB <- with(df,impute(JOB,max))
df$CLAGE <- with(df,impute(CLAGE,mean))
df$CLNO <- with(df,impute(CLNO,mean))

In [4]:
# Remove remaining missing values
df <- na.omit(df)

In [5]:
# Select numeric variables
df_processed <- df[,c(1:4,7:13)]

In [6]:
# Scale numeric variables
maxs <- apply(df_processed,2,max)
mins <- apply(df_processed,2,min)

In [7]:
df_processed <- as.data.frame(scale(df_processed, center = mins, scale=maxs-mins))

In [8]:
# Manually convert categocial variables into dummy coding
df_processed <- cbind(df_processed, df$REASON == 'HomeImp')
names(df_processed)[12] <- 'REASONH'

In [9]:
df_processed <- cbind(df_processed, df$JOB == 'Mgr')
df_processed <- cbind(df_processed, df$JOB == 'Office')
df_processed <- cbind(df_processed, df$JOB == 'ProfExe')
df_processed <- cbind(df_processed, df$JOB == 'Sales')
df_processed <- cbind(df_processed, df$JOB == 'Self')

In [10]:
names(df_processed)[13] <- 'JOBM'
names(df_processed)[14] <- 'JOBO'
names(df_processed)[15] <- 'JOBP'
names(df_processed)[16] <- 'JOBSa'
names(df_processed)[17] <- 'JOBSe'

In [11]:
# Data Partition
set.seed(101)
sample <- sample.split(df_processed$BAD, SplitRatio = 0.7)
df.train = subset(df_processed, sample == TRUE)
df.valid = subset(df_processed, sample == FALSE)

In [12]:
# Train the neural net model
nn <- neuralnet(BAD~LOAN+MORTDUE+VALUE+YOJ+DEROG+DELINQ+CLAGE+NINQ+CLNO+DEBTINC+REASONH+JOBM+JOBO+JOBP+JOBSa+JOBSe,data=df.train,hidden=c(5,3),linear.output=FALSE)

In [26]:
# Visualize the neural net 
plot(nn)

In [19]:
# Compute predictions with the validation dataset
pred.probabilities <- compute(nn,df.valid[2:17])

In [20]:
# Extract predicted probabilities from the neural net computation results
pred.probabilities <- as.numeric(pred.probabilities$net.result)

In [21]:
# Create the confusion matrix
pred.Result <- ifelse(pred.probabilities>0.5,1,0)

In [22]:
print("The confusion matrix is:")
print(table(pred.Result,df.valid$BAD))

[1] "The confusion matrix is:"
           
pred.Result   0   1
          0 959  65
          1   9  35


In [23]:
detach(package:neuralnet,unload = T)

In [24]:
# Create the ROC curve
pred <- prediction(pred.probabilities,df.valid$BAD)
perf <- performance(pred, "tpr", "fpr")
plot(perf)
abline(a=0,b=1)

In [25]:
# Calculate and print AUC value
auc <- performance(pred, measure="auc")
auc <- auc@y.values[[1]]
print(paste("AUC for the baseline regression model is:", auc))

[1] "AUC for the baseline regression model is: 0.723574380165292"
